# TD2 - Prétraitement des données avec Python et `pandas`

Le but de ce deuxième TD est d'appliquer les techniques de nettoyage et de prétraitement des données (***data preprocessing***) vues en cours sur un **jeu de données réel** issu de l'**open data**.

Nous introduirons également le format dit *tidy data* et aborderons les différentes corrections classiques à effectuer afin de transformer un jeu de données vers ce format.

Ce premier cas de preprocessing est **guidé de A à Z** afin de vous permettre de vous concentrer davantage sur les techniques utilisées et les corrections appliquées que sur le code. Vous trouverez cependant à la fin de ce notebook quelques suggestions de jeux de données publics sur lesquels vous pourrez vous entraîner.

## 1. Lecture et découverte des données

### Présentations

Les données principales utilisées pour ce TD sont issues du **fichier des prénoms** en France, de 1900 à 2018, avec l'information du département de naissance. Les données se présentent sous la forme d'un fichier `.csv` téléchargeable librement sur le [site de l'INSEE](https://www.data.gouv.fr/fr/datasets/fichier-des-prenoms-edition-2016-voir-fichier-des-prenoms-de-1900-a-2019/). 

Ces données sont également présentes dans le dossier où ce situe ce notebook, au chemin `data/dpt2018.csv`

Nous allons conduire un prétraitement classique sur ces données en suivant les grandes étapes abordées en cours. Pour rappel :
1. Nettoyage
2. Intégration
3. Réduction
4. Transformation

Le jeu de données est fourni par un service public authentifié, nous pouvons donc espérer que celui-ci soit **exhaustif** (de toute façon, en l'absence d'un accès direct aux services d'état civil, il nous est impossible de vérifier cette exhaustivité).

[Une documentation](https://www.insee.fr/fr/statistiques/2540004#dictionnaire) est fournie avec ce jeu de données, elle explicite les différentes variables fournies et leur rôle. 

>Le second fichier départemental comporte **3 624 994** enregistrements et **cinq variables** décrites ci-après.
>
> Ce fichier est trié selon les variables SEXE, PREUSUEL, ANNAIS, DPT.
>
>    * Nom : **SEXE** - intitulé : sexe - Type : caractère - Longueur : 1 - Modalité : 1 pour masculin, 2 pour féminin
>
>    * Nom : **PREUSUEL** - intitulé : premier prénom - Type : caractère - Longueur : 25
>
>    * Nom : **ANNAIS** - intitulé : année de naissance - Type : caractère - Longueur : 4 - Modalité : 1900 à 2018, XXXX
>
>    * Nom : **DPT** - intitulé : département de naissance - Type : caractère - Longueur : 3 - Modalité : liste des départements, XX
>
>    * Nom : **NOMBRE** - intitulé : fréquence - Type : numérique - Longueur : 8


Nous avons donc affaire à 5 variables dont 3 qualitatives (`sexe`, `preusuel` et `dpt`). Un certain nombre de contraintes sont précisées dans cette documentation, nous pourrons donc les vérifier pour nous assurer de l'intégrité des données téléchargées.

### Petite remarque sur l'encodage des fichiers

En Python, toutes les chaînes de caractères en mémoire sont automatiquement encodées en UTF-8 par défaut. Cependant les fichiers sur votre machine ne sont pas nécessairement écrits avec cet encodage.

Plutôt que d'improviser, Python va s'adapter à l'encodage par défaut spécifié par votre machine et tenter d'ouvrir les fichiers avec cet encodage. Vous pourrez bien sûr préciser un autre encodage lors de la lecture si vous le connaissez.

Vous pouvez accéder à cet encodage par défaut avec la cellule ci-dessous.

In [1]:
import locale
locale.getpreferredencoding()

'cp1252'

Sur une machine Windows 10 avec la langue d'affichage française, cet encodage est `cp1252`, une extension de l'encodage classique `latin-1` et limitée en terme de représentation d'accents et autres caractères diacritiques.

Si la cellule précédente vous renvoie `utf-8`, les quelques cellules ci-dessous ne s'appliqueront pas à vous. Néanmoins je vous recommande de les lire, vous allez forcément tomber un jour sur un problème d'encodage (en Python ou ailleurs).

Nous allons tenter de lire le fichier de données sans spécifier d'encodage, Python va donc utiliser celui renvoyé dans la cellule ci-dessus. Lisons uniquement les 10 premières lignes.

In [2]:
# la syntaxe with(...) est un context manager. Elle permet ici de fermer automatiquement 
# le fichier à la fin de la lecture

with open('data/dpt2018.csv', mode='r') as f:
    for i in range(10):
        print(f.readline(), end='')

ï»¿sexe;preusuel;annais;dpt;nombre
1;A;XXXX;XX;27
1;AADAM;XXXX;XX;27
1;AADEL;XXXX;XX;55
1;AADIL;1983;84;3
1;AADIL;1992;92;3
1;AADIL;XXXX;XX;175
1;AAHIL;2016;95;3
1;AAHIL;XXXX;XX;17
1;AAKASH;XXXX;XX;26


Plusieurs observations se dégagent :
* Le fichier semble être un **fichier CSV** classique mais son séparateur est un **point-virgule** (la fameuse exception culturelle française certainement).
* On retrouve bien les **5 variables** mentionnées dans la documentation, avec les mêmes noms qui plus est. La documentation récupérée est donc correcte et à jour.
* On observe bien les modalités correspondantes à des données manquantes mentionnées dans la documentation :
    * `XX` pour `dpt`
    * `XXXX` pour `annais`
* L'encodage par défaut (`cp1252` dans mon cas) provoque un problème de lecture illustré par des caractères étranges au début de la première ligne (`ï»¿`).

L'encodage par défaut de ma machine **ne correspond** donc **pas à celui du fichier**, il est alors nécessaire de **trouver l'encodage du fichier**.

Détecter automatiquement l'encodage correct d'un fichier est en réalité un problème particulièrement épineux, surtout pour des encodages exotiques. Des librairies comme [chardet](https://github.com/chardet/chardet) essaient de résoudre ce problème mais ne fonctionnent pas toujours.

En pratique, il vaut mieux simplement trouver une documentation sur la source qui peut indiquer le format des fichiers ou demander des précisions à l'émetteur. Pas de chance, cet encodage n'est **pas précisé dans la** [**documentation**](https://www.insee.fr/fr/statistiques/2540004#documentation) ! 

Avant de commencer à tester tous les encodages présents dans l'univers (je vous laisse apprécier [le nombre](https://www.iana.org/assignments/character-sets/character-sets.xhtml)), nous allons tout de même tenter de lire le fichier avec l'encodage **`UTF-8`**. En règle générale, tout système moderne bien configuré (tel Python) est censé émettre par défaut du contenu en `UTF-8`. Cet encodage possède en effet le bon goût de supporter virtuellement tous les symboles utilisés à travers la planète (environ 138 000 caractères nommés à date).

On va donc supposer que les statisticiens de l'INSEE ont adopté des bonnes pratiques et utilisé l'encodage `UTF-8`.

In [3]:
with open('data/dpt2018.csv', mode='r', encoding='utf-8') as f: # On précise explicitement l'encodage cette fois-ci
    for i in range(10):
        print(f.readline(), end='')

﻿sexe;preusuel;annais;dpt;nombre
1;A;XXXX;XX;27
1;AADAM;XXXX;XX;27
1;AADEL;XXXX;XX;55
1;AADIL;1983;84;3
1;AADIL;1992;92;3
1;AADIL;XXXX;XX;175
1;AAHIL;2016;95;3
1;AAHIL;XXXX;XX;17
1;AAKASH;XXXX;XX;26


L'encodage semble correct, les caractères étranges en début de chaîne ont disparu. Pour ceux voulant s'intéresser davantage à ce problème, ces caractères étaient en réalité une représentation du [Byte Order Mark](https://fr.wikipedia.org/wiki/Indicateur_d%27ordre_des_octets), typique des encodages Unicode.

### Lecture des fichiers

Passé l'étape de la découverte de l'encodage, nous pouvons maintenant lire le fichier avec `pandas`.

In [4]:
import pandas as pd
import numpy as np

df = pd.read_csv('data/dpt2018.csv', 
                 encoding='utf-8', # On précise notre encodage
                 sep=';',  # Séparateur non standard, nous le précisons
                 na_values=['XXXX', 'XX']) # On force pandas à reconnaître ces valeurs comme manquantes

df.head()

,sexe,preusuel,annais,dpt,nombre
0,1,A,NaN,NaN,27
1,1,AADAM,NaN,NaN,27
2,1,AADEL,NaN,NaN,55
3,1,AADIL,1983.0,84.0,3
4,1,AADIL,1992.0,92.0,3


La lecture s'est déroulée sans encombre. Nous nous assurons que les types de données sont corrects pour chaque colonne.

In [5]:
df.dtypes

sexe          int64
preusuel     object
annais      float64
dpt         float64
nombre        int64
dtype: object

`pandas` ne définira jamais de lui-même de variable qualitative mais trouvera un type quantitatif approprié (par exemple `int64` pour `sexe` alors qu'il s'agit d'une variable binaire).

Nous transformons les données vers le bon type à l'aide de la fonction `astype`.

In [6]:
df = df.astype({'sexe': 'category',
                'preusuel': 'category',
                'annais': 'Int64',
                'dpt': 'Int64', # Département d'abord converti en entier
                'nombre': 'Int64'})

df = df.astype({'dpt': 'category'}) # puis en variable qualitative

print(df.dtypes)
df.head()

sexe        category
preusuel    category
annais         Int64
dpt         category
nombre         Int64
dtype: object


,sexe,preusuel,annais,dpt,nombre
0,1,A,<NA>,NaN,27
1,1,AADAM,<NA>,NaN,27
2,1,AADEL,<NA>,NaN,55
3,1,AADIL,1983,84,3
4,1,AADIL,1992,92,3


Voilà qui est mieux, nous pouvons enfin commencer la phase de prétraîtement.

## 2. Nettoyage

Commençons par vérifier la volumétrie des données.

In [7]:
print('Nombre total de lignes :', len(df))
print('Nombre total de prénoms uniques :', len(df.preusuel.unique()))
print('Nombre total de départements uniques :', len(df.dpt.unique()))
print('Nombre total de naissances enregistrées : ', df.nombre.sum())

Nombre total de lignes : 3624994
Nombre total de prénoms uniques : 33484
Nombre total de départements uniques : 100
Nombre total de naissances enregistrées :  85139389


In [8]:
# Comptons le nombre de valeurs uniques dans chaque colonne. Les valeurs nulles sont ignorées par défaut.
df.nunique()

sexe            2
preusuel    33483
annais        119
dpt            99
nombre       2732
dtype: int64

Vérifions ce volume avec les informations de la documentation et nos connaissances "métier" (en l'occurence connaissance de la géographie française) :
* 3 624 994 enregistrements au total comme prévu
* 119 années distinctes : de 1900 à 2018
* 99 départements contre 101 théoriquement : la documentation précise que la Corse compte pour un département (n° 20) et que Mayotte n'est pas prise en compte. On retrouve bien 101 - 2 == 99 modalités possibles (Remarque : la documentation indique également que les DOM sont aussi réunis (971, 972, 973 et 974 → 97), en pratique ce n'est pas le cas. **Toujours prendre les documentations avec des pincettes**)
* 2 sexes distincts : Masculin / Féminin (seul le sexe biologique de l'enfant est pris en compte)

Le fichier contient des informations sur **33484 prénoms différents** (en moyenne moins de 300 nouveaux prénoms introduits chaque année, y compris en comptant les prénoms composés). Ce nombre paraît faible au premier abord, cependant la documentation du fichier nous indique quelques critères que chaque prénom doit remplir pour posséder sa propre ligne :
>    1. Sur la période allant de 1900 à 1945, le prénom a été attribué au moins 20 fois à des personnes de sexe féminin et/ou au moins 20 fois à des personnes de sexe masculin dans le département
>    2. Sur la période allant de 1946 à 2018, le prénom a été attribué au moins 20 fois à des personnes de sexe féminin et/ou au moins 20 fois à des personnes de sexe masculin dans le département
>    3. Pour une année de naissance donnée, le prénom a été attribué au moins 3 fois à des personnes de sexe féminin ou de sexe masculin dans le département

Ces mesures permettent de garantir un certain anonymat pour les personnes possédant des prénoms rares. Les effectifs de ces prénoms rares (i.e. ceux ne respectant pas les critères au-dessus) sont indiqués de deux façons :
* Si le critère 1 et 2 sont pas respectés : agrégés par sexe et par année dans la modalité `_PRENOMS_RARES` de `preusuel`.
* Si le critère 2 est respecté mais pas le critère 3 (prénom courant historiquement, mais rare lors d'une année dans un département) : le prénom possède sa propre ligne avec une modalité dans `preusuel` mais `annais` et `dpt` sont manquants.

### Gestion des valeurs manquantes

Mesurons désormais le nombre de lignes manquantes dans les données par variable.

In [9]:
df.isna().sum()

sexe            0
preusuel        1
annais      35608
dpt         35608
nombre          0
dtype: int64

Deux observations :
* Il existe autant de valeurs manquantes dans `annais` que dans `dpt`. Ces valeurs manquantes correspondent sûrement à la **mesure d'anonymisation** présentée ci-dessus. Nous vérifierons la **co-occurence** de ces valeurs manquantes pour s'en assurer.
* Il existe une ligne vide dans `preusuel`

Intéressons nous à cette fameuse ligne vide

In [10]:
df[df.preusuel.isna()]

,sexe,preusuel,annais,dpt,nombre
3167492,2,NaN,<NA>,NaN,31


La documentation n'indique pas l'existence possible d'un prénom vide. Il peut s'agir soit d'une erreur technique, soit d'un cas ambigu comme une naturalisation d'un enfant (certaines cultures permettent de ne pas avoir de prénom).

Dans tous les cas, compte tenu de l'effectif réduit de cette ligne au regard de l'effectif total, nous préférons se séparer de cette ligne.

In [11]:
df = df[~df.preusuel.isna()] # L'opérateur ~ permet d'inverser un filtre booléen de sélection (opérateur NOT)

# On vérifie que la ligne a été supprimée
df.isna().sum()

sexe            0
preusuel        0
annais      35607
dpt         35607
nombre          0
dtype: int64

Vérifions maintenant la co-occurrence des données manquantes entre les colonnes `annais` et `dpt`. On veut s'assurer qu'il n'existe aucune ligne où seule une des deux colonnes est manquante.

In [12]:
df[(df.annais.isna() & ~df.dpt.isna()) | (~df.annais.isna() & df.dpt.isna())]

,sexe,preusuel,annais,dpt,nombre


Il y a donc une co-occurrence parfaite, on peut estimer que ces informations ont donc été masquées comme expliqué précédemment.

Nous sommes désormais face à un dilemme :
* Les lignes où `annais` et `dpt` sont manquants représentent **presque 10% du nombre total de lignes**
* Mais en l'absence de ces informations, il est **particulièrement difficile d'analyser ces lignes** (ce qui est bien tout l'intérêt de la méthode d'anonymisation)

Nous ne voulons donc pas perdre autant d'information, mais nous ne pouvons pas l'analyser pour autant.

Nous pourrions **tenter de dé-anonymiser** ces données. Pour ce faire, il faudrait apprendre la **distribution conjointe** $ P(prenom, annee, departement) $ sur les données existantes puis effectuer une inférence pour les lignes où `annais` et `dpt` sont manquants pour retrouver des valeurs plausibles. Il existe également un indice : on sait que pour chaque case manquante, l'effectif doit forcément être inférieur strictement à 3 (sinon il ne serait pas anonymisé). Cette démarche sera donc assez efficace puisqu'il sera difficile de trop se tromper et donc de rajouter du bruit dans les données.

Le sujet de la dé-anonymisation sera abordé lors du dernier CM si le temps le permet. Pour l'instant, nous adopterons une méthode plus simple  : les effectifs des lignes avec `dpt` et `annais` manquants vont être intégrées (proportionnellement) dans les effectifs de `preusuel = _PRENOMS_RARES`. En effet, on peut bien considérer que ces prénoms sont **localement et temporairement** rares. 

In [13]:
# Commençons par compter les effectifs par sexe pour les lignes anonymisées
missing_values_by_sex = df[df.annais.isna()].groupby('sexe').nombre.sum()

# On souhaite répartir équitablement ces effectifs dans ceux de la modalité _PRENOMS_RARES
# Pour ce faire, nous allons choisir comme poids de répartition les fréquences de cette modalité, calculées
# sur la base de l'effectif total de la modalité pour chaque sexe
# Autrement dit, la somme des poids doit valoir 1 pour les hommes et 1 pour les femmes

weights = df[(df.preusuel == '_PRENOMS_RARES') & df.annais.notna()].copy()
effectifs_rares_par_sexe = weights.groupby(['sexe']).nombre.sum() # Bases de calcul des fréquences

weights['freq'] = weights.nombre / effectifs_rares_par_sexe.loc[weights.sexe].values
# Remarque : le .values est nécessaire pour effectuer l'opération élément par élément
# En effet, weights et effectifs_rares_par_sexe n'ont pas le même index
# Essayez de refaire l'opération en supprimant le .values !

print(missing_values_by_sex)
weights.head()

sexe
1    3684762
2    4576895
Name: nombre, dtype: Int64


,sexe,preusuel,annais,dpt,nombre,freq
1666882,1,_PRENOMS_RARES,1900,2,7,0.000009
1666883,1,_PRENOMS_RARES,1900,4,9,0.000012
1666884,1,_PRENOMS_RARES,1900,5,8,0.000011
1666885,1,_PRENOMS_RARES,1900,6,23,0.00003
1666886,1,_PRENOMS_RARES,1900,7,9,0.000012


In [14]:
# Vérifions que nos poids soient bien calculés
weights.groupby('sexe').freq.sum()

sexe
1    1.0
2    1.0
Name: freq, dtype: Float64

In [15]:
# On calcule enfin les nouveaux effectifs de la modalité "_PRENOMS_RARES"
# On s'assure que les effectifs rajoutés sont entiers
weights.nombre = weights.nombre + (weights.freq * missing_values_by_sex.loc[weights.sexe].values).astype('int')

In [16]:
# Vérifions que les effectifs totaux rajoutés correspondent bien aux effectifs totaux des lignes avec
# des données manquantes

print('Effectif totaux des lignes avec des données manquantes : ', df[df.annais.isna()].nombre.sum())
print('Effectif totaux rajoutés à la modalité "_PRENOMS_RARES" : ',
      weights.nombre.sum() - df[(df.preusuel == '_PRENOMS_RARES') & df.annais.notna()].nombre.sum())

Effectif totaux des lignes avec des données manquantes :  8261657
Effectif totaux rajoutés à la modalité "_PRENOMS_RARES" :  8251153


La différence entre les effectifs initiaux et les effectifs rajoutés s'explique par l'opération de conversion vers des entiers (tronquage) dans la cellule précédente.

Il serait possible de s'assurer que les effectifs rajoutés correspondent parfaitement mais cela complexifierait cette étape. Nous nous contenterons de ce résultat approximatif, l'erreur commise étant dans tous les cas insignifiante face aux effectifs totaux de la modalité `_PRENOMS_RARES`.

Nous remplaçons les effectifs ainsi calculés dans `df`.

In [17]:
df.loc[weights.index, 'nombre'] = weights.nombre

Nous pouvons désormais supprimer les lignes anonymisées, leurs effectifs ont été réattribués.

In [18]:
df = df[df.annais.notna()]

df.isna().sum() # Vérification de notre travail de nettoyage

sexe        0
preusuel    0
annais      0
dpt         0
nombre      0
dtype: int64

Les valeurs manquantes étant maintenant éradiquées / redistribuées, nous nous attaquons à la **détection des valeurs aberrantes**.

### Détection des valeurs aberrantes

Le jeu de données est livré avec un **ensemble de contraintes fortes** rappelées ci-dessous. Nous vérifions ces contraintes.

Contraintes issues de la documentation :
* Longueur maximale pour `preusuel` : 25 caractères
* Aucune espace permise dans `preusuel`, seules les lettres et le symbole `-` sont tolérés
* L'effectif minimal possible pour un quadruplet `(sexe, prénom, année, département)` est de 3 (sans quoi les données auraient été anonymisées)
* Pour chaque modalité de `preusuel` autre que `_PRENOMS_RARES` et pour chaque département, nous devons avoir un effectif total d'au moins 20 naissances de 1900 à 1945 et également 20 naissances de 1946 à 2018.

N'hésitez pas à séparer la cellule suivante en plusieurs cellules pour bien comprendre les étapes de calcul. La vérification de contraintes d'intégrité comme celles-ci est une étape importante pour évaluer la qualité des données, en Python comme ailleurs. 

In [20]:
# Première contrainte : longueur max du prénom = 25 caractères

print('Longueur du prénom le plus long (doit être < 25 caractères) : ', df.preusuel.str.len().max())

# Deuxième contrainte : caractères permis dans preusuel = lettres + "-"
# On utilise une regex pour valider ce format
# Attention : les prénoms enregistrés sont accentués, nous devons accepter les accents
# En cherchant un peu, la liste des caractères autorisés est la suivante :
# A-Z , À, Â, Ä, Ç, É, È, Ê, Ë, Î, Ï, Ô, Ö, Ù, Û, Ü, Ÿ, Æ, Œ (+ équivalents en minuscule)
# Trait d'union et apostrophe

print('Nombre de prénoms contenant un caractère interdit : ',
      len(df[~df.preusuel.str.contains("[-'A-ZÀÂÄÇÉÈÊËÎÏÔÖÙÛÜŸÆŒ]+", regex=True)]))

# Troisième contrainte : effectif minimal pour un même quadruplet  >= 3
print('Effectif minimal pour un quadruplet (sexe, département, année, prénom) : ', df.nombre.min())

# Quatrième contrainte
# Plus délicate à vérifier, nous devons agréger les données par prénom, sexe et département ainsi que par plage
# de date (de 1900 à 1945, de 1946 à 2018). L'agrégation sera une somme sur les effectifs
# Nous allons créer une colonne booléenne qui indique si la ligne concerné

check = df[df.preusuel != '`_PRENOMS_RARES'].copy()
check_gb = check.groupby(['sexe', 'dpt', 'preusuel', (df.annais <= 1945)], observed=True).nombre.sum()

print("Nombre de prénoms ne respectant pas les critères d'anonymisation : ", len(check_gb[check_gb < 20]))

Longueur du prénom le plus long (doit être < 25 caractères) :  18
Nombre de prénoms contenant un caractère interdit :  0
Effectif minimal pour un quadruplet (sexe, département, année, prénom) :  3
Nombre de prénoms ne respectant pas les critères d'anonymisation :  126534


Visiblement la dernière condition n'est pas réellement respectée. Affichons quelques modalités au hasard ne respectant pas cette condition.

Rappelez vous que dans la sortie suivante la colonne `annais` contient un booléen valant True si `annais <= 1945`, False sinon.

In [21]:
check_gb[check_gb < 20].sample(25)

sexe  dpt  preusuel    annais
1     5    GRÉGOIRE    False      3
2     45   FATMA       False     19
1     37   BRIEUC      False      4
      93   LENI        False      7
      1    ISMAEL      False      3
2     80   MEGAN       False      3
1     50   PIERRE-LUC  False      3
2     83   ANOUCK      False      3
1     974  RUSSEL      False     11
2     60   CLELIE      False      3
      972  MARILIA     False      3
      22   GAEL        False      6
      75   DORIENNE    False      3
      84   HAYAT       False     18
      59   LUDYVINE    False      3
      69   SAJIDA      False     12
1     59   ALVYN       False     11
2     83   WARDA       False      3
1     59   SANTO       False      3
      15   EMILIEN     False     12
      69   SOPHIEN     False      8
2     78   CARINNE     False      9
      55   SHIRLEY     False      6
      30   JANIE       True       6
      86   MARIKA      False      3
Name: nombre, dtype: Int64

L'**anonymisation** des données n'est donc **pas tout à fait rigoureuse** (ou les règles précisées ne correspondent pas exactement à celles appliquées lors de la publication du jeu de données). A nouveau, nous voyons que la documentation peut nous guider, mais il est dangereux de la suivre aveuglément dans la plupart des cas.

Ces lignes restent toutefois valides, donc nous n'allons bien sûr pas les supprimer.

#### Analyse univariée / Profilage des données

Pour identifier désormais de façon statistique les valeurs aberrantes, nous procédons à une analyse univariée de chaque variable du tableau.

Nous allons utiliser la bibliothèque `pandas_profiling` qui réalise automatiquement la plupart des opérations nécessaires et présente les résultats clairement. 

Un bon exercice serait néanmoins de recalculer la plupart des indicateurs numériques manuellement à l'aide de `pandas`.

In [23]:
import sys
!{sys.executable} -m pip install pandas-profiling
from pandas_profiling import ProfileReport

report = ProfileReport(df, minimal = True) # Difficile de faire plus simple, minimal = True pour alléger le calcul

# Vous pouvez également enregistrement une copie statique de ce rapport avec la ligne suivante
# report.to_file(output_file = 'prenom_profiling.html')

report

  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27085 sha256=c8620872a6241cccb3700c3198b6e75001ee825142cdcc8135e3ada39f4d91a2
  Stored in directory: c:\users\aarcuri\appdata\local\pip\cache\wheels\23\14\6e\4be5bfeeb027f4939a01764b48edd5996acf574b0913fe5243
  Created wheel for imagehash: filename=ImageHash-4.2.1-py2.py3-none-any.whl size=295198 sha256=d6818a5712cc765b5e9e6c101f2e76c0213501c84fd2e03986da4a34fe2a6f4f
  Stored in directory: c:\users\aarcuri\appdata\local\pip\cache\wheels\48\a1\7f\096c1269d6bf78d4768180602579b35a1e8cb1250bb4b40c74
Successfully built htmlmin imagehash


Summarize dataset:   0%|          | 0/14 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Quelques remarques pêle-mêle sur ce rapport
* L'index est bel et bien une **clé primaire** pour ce jeu de données, chaque valeur est unique
* La **modalité femme** apparamment **plus souvent que la modalité homme**. Ceci pourrait s'expliquer par deux facteurs, vérifiés plus tard dans l'analyse :
    * Ces effectifs sont liés au **nombre de naissances enregistrées**. Il y a naturellement plus de filles qui naissent que de garçons et statistiquement parlant : plus de naissances = plus de prénoms uniques attribués, d'où l'écart constaté
    * Il peut exister une **plus grande diversité de prénoms féminins** que masculins, ce qui engendre un nombre plus élevé de lignes avec la modalité `2`.
* Les **modalités les plus courantes pour `preusuel`** sont **cohérentes**. Camille, Marie, Pierre et Claude correspondent en effet à des prénoms réellements courants, et les proportions constatées sont plausibles. Ces prénoms étant généralement mixtes, il est également normal qu'ils apparaissent davantage que des prénoms non-mixtes. Naturellement, la modalité `_PRENOMS_RARES` est la plus courante de toutes, encore une fois avec une proportion raisonnable.
* Il y a visiblement de plus en plus de lignes au fil des années. Ceci peut s'expliquer par une **croissance constante du nombre de prénoms distincts donnés chaque année**. Là encore, les valeurs ne sont pas choquantes et semblent correspondre à la réalité.
* Les **départements** possédant le plus de lignes associées sont bien ceux qui sont **les plus peuplés** (et enregistrent le plus grands nombre de naissances) : Paris, Nord (Lille), Rhône (Lyon), Bouches-du-Rhône (Marseille) et Pas de Calais. Les valeurs de la variable `dpt` paraîssent cohérentes.
* La distribution des effectifs est **très nettement biaisée** (*skewed*) **vers la gauche**. Ceci paraît également cohérent, pour un même triplet `(sexe, année, département)`, il paraît peu probable qu'un nombre important de naissances sont enregistrées pour un prénom donné. On observe que 95% des effectifs sont inférieurs ou égaux à 84 naissances ce qui semble acceptable. Les valeurs maximales sont néanmoins assez élevées, intéressons nous à elles.

In [24]:
# Lignes avec les plus grands effectifs
df.nlargest(n=15, columns=['nombre'])

,sexe,preusuel,annais,dpt,nombre
3621334,2,_PRENOMS_RARES,2017,973,12193
3621433,2,_PRENOMS_RARES,2018,973,11802
3621428,2,_PRENOMS_RARES,2018,93,11583
3621410,2,_PRENOMS_RARES,2018,75,11418
3621131,2,_PRENOMS_RARES,2015,93,11384
3621311,2,_PRENOMS_RARES,2017,75,11000
3620717,2,_PRENOMS_RARES,2011,75,10954
3621329,2,_PRENOMS_RARES,2017,93,10940
3621230,2,_PRENOMS_RARES,2016,93,10927
3621032,2,_PRENOMS_RARES,2014,93,10907


In [25]:
# Comparons ces nombres de naissances aux effectifs totaux pour ces quelques départements et années
df[df.dpt.isin([973, 93, 75]) & 
   (df.annais >= 2014) & 
   (df.sexe == 2)].groupby(['sexe', 'annais', 'dpt'], observed=True).nombre.sum()

sexe  annais  dpt
2     2014    75     26954
              93     18977
              973    10060
      2015    75     27048
              93     19384
              973    10547
      2016    75     26888
              93     19085
              973    11424
      2017    75     26961
              93     18867
              973    13082
      2018    75     27783
              93     19232
              973    12677
Name: nombre, dtype: Int64

Les lignes avec des effectifs élevés sont 
1. Celles correspondant aux prénoms rares
2. Récentes (2014 et plus)
3. Issues des départements avec un fort taux de natalité pour ces années (Paris, Seine St Denis et Guyane)
4. Correspondent à uniquement à des naissances de filles (`sexe = 2`)

Plusieurs raisons pourraient expliquer ces effectifs pour cette modalité en particulier :
* Explosion du nombre de prénoms uniques ces dernières années (variations orthographiques, nouveaux prénoms, prénoms composés, ...)
* Possiblement un grand nombre de prénoms ethniques insuffisamment attribués historiquement (au moins 40 attributions historiques d'après les règles d'anonymisation)

Ces effectifs sont néanmoins considérables, représentants souvent plus de 80% des naissances enregistrées dans le département cette année-là.

Nous étudierons ce phénomène plus en détail dans l'analyse. Il est difficile de voir en l'état si ces valeurs sont aberrantes ou non. Dans tous les cas de figure, la modalité `_PRENOMS_RARES` devra forcément être traitée à part compte tenu de sa nature spéciale (modalité générique).

En somme, à la sortie de cette analyse préliminaire, nous n'avons pas pu identifier de valeurs singulièrement aberrantes. Nous pouvons passer à la suite du prétraitement.

La phase de nettoyage se termine ici. Néanmoins, en cas de découverte d'un problème plus tard dans l'analyse, il sera nécessaire de compléter cette étape. On observe donc le **caractère itératif du processus KDD**. 

Avant de passer à l'étape d'intégration, nous allons toutefois prendre soin de renommer quelques modalités pour être plus explicite avant de **sauvegarder une copie des données nettoyées** !

In [26]:
# Renommage de modalités et de variables

df = df.rename(columns={'preusuel': 'prenom', 'annais': 'annee', 'dpt': 'departement'})
df.sexe = df.sexe.cat.rename_categories({1: 'Homme', 2: 'Femme'})

print(df.head())

# Sauvegarde des données
df.to_csv('dpt_2018_clean.csv')

     sexe prenom  annee departement  nombre
3   Homme  AADIL   1983          84       3
4   Homme  AADIL   1992          92       3
6   Homme  AAHIL   2016          95       3
9   Homme  AARON   1962          75       3
10  Homme  AARON   1976          75       3


----

## 3. Intégration

Commençons par recharger le jeu de données nettoyées pour pouvoir partir de ce point en cas de redémarrage du notebook.

In [27]:
import pandas as pd

df = pd.read_csv('dpt_2018_clean.csv', index_col = 0) # Utiliser la première colonne du fichier comme index

C:\Users\aarcuri\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Le fichier des prénoms de l'INSEE nettoyé représente d'ores et déjà une **mine d'informations**. Il permet en effet de s'intéresser à de nombreux points, certains sérieux d'autres moins. Outre l'aspect des prénoms, ce fichier permet en effet de **retracer précisément la natalité en France** et d'identifier les **évolutions démographiques** engendrées. En outre, les 119 ans d'historique dans ce fichier permettent également d'observer l'**impact d'évènements marquants** du XXe siècle tels les deux **guerres mondiales**.

Afin de pouvoir se donner encore plus de possibilités d'analyse, nous allons choisir de **fusionner ce fichier avec une deuxième** [**source**](https://github.com/MatthiasWinkelmann/firstname-database). Cette source fournit pour un grand nombre de prénoms (> 40000) :
* la fréquence d'attribution dans différentes régions du monde 
* le genre généralement associé au prénom (sur une échelle plus fine que simplement Homme/Femme)

Cette seconde source permettra d'une part de **comparer les pratiques de nommage françaises avec celle du reste du monde** (par exemple : existent-ils des prénoms traditionnellement masculins en France mais généralement féminins à l'échelle du monde entier) ainsi que les **écarts de fréquence d'attributions**. On s'attend bien sûr sur ce dernier point à retrouver des noms "franchouillards" à être drastiquement sur-représentés en France mais nous ne sommes pas à l'abri d'une bonne surprise. Enfin, cette seconde source associant à chaque prénom une distribution géographique (**très approximative**), nous pourrons peut-être espérer retrouver dans les données une manifestation des [différentes vagues d'immigration](https://fr.wikipedia.org/wiki/Histoire_de_l'immigration_en_France) qui ont ponctué le XXe siècle.

Quelques précautions s'imposent tout de fois :
* Ce second fichier trouve son origine dans un travail collectif organisé par un [magazine allemand](https://www.heise.de/ct/ftp/07/17/182/), il y a donc naturellement un **biais sur la qualité des données** envers les prénoms occidentaux (qui composent vraisemblablement une bonne partie du lectorat)
* Les **variations orthographiques** entre les prénoms peuvent bruiter l'analyser (e.g. `Tatiana`, `Tatyana`, `Tetyana`, `Tatsiana` sont des variantes courantes d'un même prénom)
* Le fichier des prénoms ne fait qu'enregistrer les prénoms transmis à la naissance à l'état civil, il n'est **pas forcément représentatif** de la distribution des prénoms des personnes vivant en France.

Au final, l'important est surtout d'apprendre à manipuler des données sur un phénomène compréhensible par tout le monde. Gardez donc simplement à l'esprit que **les résultats que vous obtenez peuvent ne pas refléter exactement la réalité**.

### Lecture, interprétation, nettoyage

Vous pouvez télécharger le fichier dans [ce dépot Github](https://github.com/MatthiasWinkelmann/firstname-database). Il est également présent dans ce répertoire au chemin **`data/firstnames_database.csv`**.

Nous allons passer rapidement sur l'étape de nettoyage de ce fichier, elle est similaire à l'étape vue précédemment sur le fichier des prénoms.

In [28]:
# Lecture
prenom_db = pd.read_csv('data/firstnames_database.csv', header=0, sep=';', encoding='utf-8')

print(prenom_db.columns)

prenom_db.sample(10)

Index(['name', 'gender', 'Great Britain', 'Ireland', 'U.S.A.', 'Italy',
       'Malta', 'Portugal', 'Spain', 'France', 'Belgium', 'Luxembourg',
       'the Netherlands', 'East Frisia', 'Germany', 'Austria', 'Swiss',
       'Iceland', 'Denmark', 'Norway', 'Sweden', 'Finland', 'Estonia',
       'Latvia', 'Lithuania', 'Poland', 'Czech Republic', 'Slovakia',
       'Hungary', 'Romania', 'Bulgaria', 'Bosnia and Herzegovina', 'Croatia',
       'Kosovo', 'Macedonia', 'Montenegro', 'Serbia', 'Slovenia', 'Albania',
       'Greece', 'Russia', 'Belarus', 'Moldova', 'Ukraine', 'Armenia',
       'Azerbaijan', 'Georgia', 'Kazakhstan/Uzbekistan,etc.', 'Turkey',
       'Arabia/Persia', 'Israel', 'China', 'India/Sri Lanka', 'Japan', 'Korea',
       'Vietnam', 'other countries'],
      dtype='object')


,name,gender,Great Britain,Ireland,U.S.A.,Italy,Malta,Portugal,Spain,France,...,"Kazakhstan/Uzbekistan,etc.",Turkey,Arabia/Persia,Israel,China,India/Sri Lanka,Japan,Korea,Vietnam,other countries
4440,Beniamino,M,NaN,NaN,NaN,-6.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37309,Sólbjört,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16634,Herfriede,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4175,Basilina,F,NaN,NaN,NaN,NaN,NaN,-8.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27219,Mevsud,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15982,Harbhajan,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-6.0,NaN,NaN,NaN,NaN
28314,Mukhayyo,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13414,Garan,F,-8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31289,Petron,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35180,Seead,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Les colonnes de ce fichier sont les suivantes :
* `name` : prénom concerné (qualitatif)
* `gender` : le genre généralement attribué (qualitatif). Modalités possibles :
    * `F` féminin
    * `1F` féminin s'il s'agit du premier prénom, sinon généralement masculin
    * `?F` généralement féminin (moins certain que `F`)
    * `M` masculin
    * `1M` masculin s'il s'agit du premier prénom, sinon généralement féminin
    * `?M` généralement masculin (moins certain que `M`)
    * `?` mixte
* Toutes les autres colonnes : fréquence d'attribution dans la région concernée. La fréquence indiquée est le $ log_2 $ de la fréquence en pourcentage. Exemple : si on lit `-3`, alors la fréquence d'attribution est $ 2^{-3} = 0.125\% = 0.00125 $

Nous allons diminuer la granularité des fréquences en regroupant ces colonnes dans les zones géographiques. Ces zones sont [celles utilisées par l'ONU](https://unstats.un.org/unsd/methodology/m49/) pour ses publications statistiques :
* `France` → `France` (à part puisque l'on s'intéresse aux fichiers des prénoms français)
* `Europe Est` → `Poland`, `Czech Republic`, `Slovakia`, `Hungary`, `Romania`, `Bulgaria`, `Russia`, `Belarus`, `Moldova`, `Ukraine`
* `Europe Nord` → `Great Britain`, `Ireland`, `Iceland`, `Denmark`, `Norway`, `Sweden`, `Finland`, `Estonia`, `Latvia`, `Lithuania`
* `Europe Sud` → `Italy`, `Malta`, `Portugal`, `Spain`, `Bosnia and Herzegovina`, `Croatia`, `Kosovo`, `Macedonia`, `Montenegro`, `Serbia`, `Slovenia`, `Albania`, `Greece`
* `Europe Ouest` → `Belgium`, `Luxembourg`, `the Netherlands`, `East Frisia` (en Allemagne moderne), `Germany`, `Austria`, `Swiss`
* `Asie Ouest` → `Armenia`, `Georgia`, `Azerbaijan`, `Turkey`, `Arabia/Persia`, `Israel`
* `Asie Centre` → `Kazakhstan/Uzbekistan,etc.`
* `Asie Est` → `China`, `India/Sri Lanka`, `Japan`, `Korea`, `Vietnam`
* `Autre` → `other countries`

Lors de l'agrégation, nous choisirons la fréquence maximale pour le prénom parmi les pays du groupe.

In [29]:
# Conversion des fréquences en pourcentage
freq_cols = prenom_db.columns[2:] # Sélection de toutes les colonnes avec une fréquence

prenom_db[freq_cols] = (2**prenom_db[freq_cols]) / 100

In [30]:
freq_cols.drop('France')

Index(['Great Britain', 'Ireland', 'U.S.A.', 'Italy', 'Malta', 'Portugal',
       'Spain', 'Belgium', 'Luxembourg', 'the Netherlands', 'East Frisia',
       'Germany', 'Austria', 'Swiss', 'Iceland', 'Denmark', 'Norway', 'Sweden',
       'Finland', 'Estonia', 'Latvia', 'Lithuania', 'Poland', 'Czech Republic',
       'Slovakia', 'Hungary', 'Romania', 'Bulgaria', 'Bosnia and Herzegovina',
       'Croatia', 'Kosovo', 'Macedonia', 'Montenegro', 'Serbia', 'Slovenia',
       'Albania', 'Greece', 'Russia', 'Belarus', 'Moldova', 'Ukraine',
       'Armenia', 'Azerbaijan', 'Georgia', 'Kazakhstan/Uzbekistan,etc.',
       'Turkey', 'Arabia/Persia', 'Israel', 'China', 'India/Sri Lanka',
       'Japan', 'Korea', 'Vietnam', 'other countries'],
      dtype='object')

In [31]:
# Regroupement par zone géographique
prenom_db['europe_est'] = prenom_db[['Poland', 'Czech Republic', 'Slovakia', 'Hungary',
                                     'Romania', 'Bulgaria', 'Russia', 'Belarus', 
                                     'Moldova', 'Ukraine']].max(axis=1) # freq max sur chaque ligne
prenom_db['europe_nord'] = prenom_db[['Great Britain', 'Ireland', 'Iceland', 'Denmark', 'Norway', 'Sweden',
                                      'Finland', 'Estonia', 'Latvia', 'Lithuania']].max(axis=1)
prenom_db['europe_sud'] = prenom_db[['Italy', 'Malta', 'Portugal', 'Spain', 'Bosnia and Herzegovina', 
                                     'Croatia', 'Kosovo', 'Macedonia', 'Montenegro', 'Serbia',
                                     'Slovenia', 'Albania', 'Greece']].max(axis=1)
prenom_db['europe_ouest'] = prenom_db[['Belgium', 'Luxembourg', 'the Netherlands', 'East Frisia',
                                       'Germany', 'Austria', 'Swiss']].max(axis=1)
prenom_db['asie_ouest'] = prenom_db[['Armenia', 'Georgia', 'Azerbaijan', 'Turkey', 
                                     'Arabia/Persia', 'Israel']].max(axis=1)
prenom_db['asie_centre'] = prenom_db[['Kazakhstan/Uzbekistan,etc.']]
prenom_db['asie_est'] = prenom_db[['China', 'India/Sri Lanka', 'Japan', 'Korea', 'Vietnam']].max(axis=1)
prenom_db['autre'] = prenom_db[['other countries']]

# Suppression des colonnes par pays (sauf celle pour la France que l'on conserve)
prenom_db = prenom_db.drop(columns=freq_cols.drop('France'))

prenom_db.head()

,name,gender,France,europe_est,europe_nord,europe_sud,europe_ouest,asie_ouest,asie_centre,asie_est,autre
0,Aad,M,NaN,NaN,NaN,NaN,0.000313,NaN,NaN,NaN,NaN
1,Aadam,M,NaN,NaN,0.000039,NaN,NaN,NaN,NaN,NaN,NaN
2,Aadje,F,NaN,NaN,NaN,NaN,0.000039,NaN,NaN,NaN,NaN
3,Aadu,M,NaN,NaN,0.000078,NaN,NaN,NaN,NaN,NaN,NaN
4,Aaf,?F,NaN,NaN,NaN,NaN,0.000039,NaN,NaN,NaN,NaN


On supprime les lignes qui n'ont aucune fréquence associée, elles ne seront d'aucune utilité. De plus, il serait risqué de tenter d'imputer des données.

In [ ]:
prenom_db = prenom_db.dropna(subset=['France', 'europe_est', 'europe_nord', 
                                     'europe_ouest', 'europe_sud','asie_ouest',
                                     'asie_centre', 'asie_est', 'autre'], how = 'all')

Vérifions le reste des valeurs nulles (en fréquence sur le nombre total de lignes)

In [ ]:
prenom_db.isna().sum() / len(prenom_db)

On supprime les quelques lignes sans prénom (sans intérêt) ainsi que la colonne `autre` qui n'apporte que peu d'informations (seulement 12 prénoms avec des données pour cette zone).

In [ ]:
prenom_db = prenom_db[prenom_db.name.notna()]
prenom_db = prenom_db.drop(columns=['autre'])

Enfin on affecte le type adéquat pour `gender`

In [ ]:
prenom_db = prenom_db.astype({'gender': 'category'})

Vérifions enfin que les valeurs contenues dans cette source paraissent cohérentes. Nous affichons les 15 noms les plus fréquents en France d'après cette source.

In [ ]:
prenom_db.nlargest(columns=['France'], n=15)

On enregistre ce fichier nettoyé et on le recharge.

In [ ]:
prenom_db.to_csv('prenom_db_clean.csv')

In [ ]:
prenom_db = pd.read_csv('prenom_db_clean.csv', index_col=0)
prenom_db = prenom_db.astype({'gender': 'category'})

### Fusion avec le fichier des prénoms (`df`)

La seule **clé de liaison** entre le fichier des prénoms (`df`) et la base d'information sur les prénoms (`prenoms_db`) est le **prénom lui-même**. Nous allons donc joindre sur cette colonne.

Cette jointure n'est pas triviale :
* Dans `prenom_db`, les séparateurs entre les prénoms (espaces, traits d'union, ...) sont représentés par le symbole `+`
* Les prénoms sont en majuscules dans `df`, sous forme Capitalisée dans `prenom_db`
* Les variantes d'un même prénom complexifient la jointure

L'idéal serait de dériver dans `df` et `prenom_db` la racine de chaque prénom sous forme standardisée et de joindre sur cette information. Il s'agit là d'une opération (dite de *stemming*) assez délicate qui relève du traîtement du langage naturel (NLP). Cette opération dépasse la portée de ce cours, nous ne l'appliquerons pas.

De façon alternative, nous pourrions simuler la phonétique de chaque nom et joindre sur la représentation phonétique. Des algorithmes comme [Metaphone](https://fr.wikipedia.org/wiki/Metaphone) permettent cette opération. Cette possibilité ne sera pas étudiée et est laissée en exercice.

Dans notre cas, nous allons simplement régler les deux premiers points de blocage listés ci-dessus. On s'attend à obtenir une **liaison `1..n`** entre `prenom_db` et `df`, nous ajouterons un test sur cette condition après la fusion.

In [ ]:
# Normalisation des séparateurs
# Dans df, seul le trait d'union est autorisé
# On remplace donc tous les séparateurs "+" dans prenom_db par des traits d'union afin de maximiser la compatibilité

prenom_db.name = prenom_db.name.str.replace('+', '-')

# Passage des prénoms dans prenom_db en majuscules
prenom_db.name = prenom_db.name.str.upper()

In [ ]:
# On fusionne prenom_db dans df

df.merge(prenom_db,
         left_on="prenom",
         right_on='name',
         how = 'left', # Equivalent à un LEFT JOIN, on souhaite garder toutes les lignes de df
         indicator = True, # Ajout de l'origine des infos fusionnées
         validate = "m:1") # Vérification de la liaison n..1 entre df et prenom_db

pandas nous informe que la relation `n..1` n'est pas vérifiée, le champ `name` n'est pas unique dans `prenom_db`.

Nous avons en effet oublié de vérifier cette propriété. En omettant le test sur la relation, la fusion aurait fonctionné mais certaines lignes auraient été dédoublées et auraient donc perturbé l'analyse en aval.

Pour supprimer les doublons dans `prenom_db`, nous allons procéder de la sorte :
* pour `gender` : conserver la modalité présentant la somme des fréquences la plus élevée (autrement dit la version la plus couramment attribuée)
* pour les fréquences : conserver le max des fréquences entre chaque ligne

In [ ]:
# Un exemple de doublon : le prénom MARIA
prenom_db[prenom_db.name == 'MARIA']

# Valeur souhaitée en sortie : gender = 1F, et toutes les fréquences remplies sauf pour asie_est

In [ ]:
# On récupère le genre le plus donné pour chaque prénom
# Pour ce faire, on calcule temporairement une colonne contenant la somme des fréquences sur chaque ligne
# On trie le tableau selon les valeurs de cette colonne, puis on groupe le tableau par "name"
# Lors de la construction du GROUP BY, pandas conserve l'ordre d'apparition des lignes dans le tableau
# Donc la ligne avec la fréquence la plus élevée sera la première de chaque groupe

prenom_db['total_freq'] = prenom_db.sum(axis=1)
prenom_db = prenom_db.sort_values('total_freq', ascending=False) # tri par ordre décroissant
top_gender = prenom_db.groupby('name').first().gender

print(top_gender['MARIA']) # On s'assure qu'on obtient bien "1F" comme prévu
top_gender.head()

In [ ]:
# On récupère pour chaque prénom le max des fréquences disponibles pour chaque zone géographique
max_freqs = prenom_db.groupby('name')[['France', 'europe_est', 'europe_nord', 
                                       'europe_sud', 'europe_ouest', 'asie_ouest', 
                                       'asie_centre', 'asie_est']].max()

max_freqs.loc['MARIA'] # On vérifie à nouveau les valeurs pour notre exemple

In [ ]:
# On supprime désormais les doublons dans le tableau d'origine et on remplace les valeurs dans le tableau dédoublonné
prenom_db = prenom_db.drop_duplicates(subset=['name'])

prenom_db.gender = top_gender[prenom_db.name].values
prenom_db[['France', 'europe_est', 'europe_nord', 
           'europe_sud', 'europe_ouest', 'asie_ouest', 
           'asie_centre', 'asie_est']] = max_freqs.loc[prenom_db.name].values

# On supprime enfin la colonne de tri "total_freq"
prenom_db = prenom_db.drop(columns=['total_freq'])

# On trie à nouveau prenom_db par ordre alphabétique sur name pour reprendre la forme d'origine
prenom_db = prenom_db.sort_values('name')
prenom_db.head()

In [ ]:
# Vérifions notre ligne exemple
prenom_db[prenom_db.name == 'MARIA']

Nous pouvons désormais retenter notre jointure précédente

In [ ]:
df = df.merge(prenom_db,
              left_on="prenom",
              right_on='name',
              how = 'left', # Equivalent à un LEFT JOIN, on souhaite garder toutes les lignes de df
              indicator = True, # Ajout de l'origine des infos fusionnées
              validate = "m:1") # Vérification de la liaison n..1 entre df et prenom_db

df.head()

La colonne `_merge` est introduite automatiquement puisque nous avons précisé le paramètre `indicator = True` lors de la fusion.

Cette colonne contient l'origine de la ligne concerné, elle peut prendre 3 modalités :
* `left_only` : la ligne n'est présente que dans le tableau de gauche
* `right_only` : la ligne n'est présente que dans le tableau de droite (impossible dans notre cas puisque nous avons fait un LEFT JOIN)
* `both` : la ligne existe dans les deux tableaux

On peut vérifier le nombre de prénoms pour lesquels des infos ont pu être fusionnées depuis `prenom_db`. Attention : on raisonne ici en nombre de modalités différentes pour `prenom`, et non en terme de nombre de lignes.

In [ ]:
df.drop_duplicates('prenom').groupby('_merge', observed=True).size()

Un peu plus de **40% des prénoms ont donc pu être caractérisés**. Comme annoncé plus haut, la jointure sur un champ mal spécifié comme un prénom **reste toujours délicate**. 

Dans le cadre de futurs prétraitement, aidez-vous autant que possible d'une **clé étrangère** (si existante) ou tentez de construire une [**clé artificielle**](https://fr.wikipedia.org/wiki/Cl%C3%A9_artificielle) dans les deux tableaux.

Pour cette première découverte, nous nous contenterons de ce résultat. Bien qu'imparfait, il peut potentiellement suffire pour des analyses pertinentes. En particulier, il est fort probable que la plupart des prénoms raisonnablement communs aient été caractérisés.

### Gestion des duplicata

Vérifions l'existence de duplicata dans le jeu de donnée. Compte tenu des données étudiées, un duplicata est un ensemble d'au moins deux lignes pour le **même prénom, même année, même département et même sexe** (un même prénom peut être unisexe après tout). 

Nous ne procèderons pas à une déduplication avancée reposant sur la fusion des entités. Il est en effet **difficile de trouver des duplicata dûs aux fautes de frappe** dans les prénoms en cela qu'une légère variation orthographique peut être voulue (e.g. "Noa" et "Noah").

In [ ]:
# La fonction drop_duplicates accepte la liste des colonnes à utiliser pour dédupliquer les données
# Dans un langage plus "SGBD", ces colonnes forment une clé artificielle, sur laquelle nous faisons un SELECT DISTINCT

old_size = len(df)
df = df.drop_duplicates(subset=['sexe', 'prenom', 'departement', 'annee'])

print(f'Nombre de lignes dupliquées retirées : {len(df) - old_size}')

Les données sont donc complètement dédupliquées dans le jeu de données initial !

Encore une fois, nous sauvegardons le jeu intégré et le rechargeons. Ces sauvegardes intermédiaires sont une **bonne pratique** à adopter pour pérenniser votre analyse, en particulier si celle-ci est **reproduite périodiquement** (e.g. pour construire un rapport à jour). En cas de pépin, vous pourrez rapidement jauger où se situe l'erreur en analysant les fichiers intermédiaires.

In [ ]:
df.to_csv('merged_prenoms.csv')
dtypes = df.dtypes

print(dtypes)

In [ ]:
import pandas as pd

df = pd.read_csv('merged_prenoms.csv', index_col=0, dtype={
    'sexe': 'category',
    'prenom': 'category',
    'departement': 'int',
    'gender': 'category',
})

df = df.astype({'departement': 'category'})


---

## 4. Réduction

Le jeu de données après nettoyage et intégration est de taille décente du haut de ses 3.5M lignes et ses 16 colonnes. En l'observant de plus près, nous nous rendons compte que de nombreuses cellules vides subsistent : le **tableau de données** est **en partie creux**.

Ces cellules vides concernent :
* les prénoms qui n'ont pas d'informations supplémentaires disponibles dans `prenom_db`. Nous ne pouvons pas faire grand chose à leur sujet et ignorons ces cellules vides
* de nombreuses **fréquences d'attribution non renseignées**

Sur ce deuxième point, nous pouvons essayer de **"compacter" ces données** en identifiant des colonnes redondantes par une analyse des corrélations, ou bien à l'aide d'une **méthode factorielle**. Puisque toutes ces fréquences sont quantitatives, la méthode adaptée sera alors une **PCA**.

Nous allons expérimenter avec les **deux techniques** dans la suite. Les colonnes concernées dans les deux cas sont celles contenant les fréquences, à savoir :
* `France`
* `europe_est`
* `europe_nord`
* `europe_sud`
* `europe_ouest`
* `asie_ouest`
* `asie_est`
* `asie_centre`

Nous allons donc sélectionner uniquement les lignes avec au moins une de ces colonnes non vide.

In [ ]:
freq_cols = ['France', 'europe_est', 'europe_nord', 
             'europe_sud', 'europe_ouest', 'asie_ouest', 
             'asie_est', 'asie_centre']

df_freq_only = df[df[freq_cols].notna().any(axis=1)]

df_freq_only.head()

#### Identification des colonnes redondantes

Commençons par tracer simplement chaque paire de colonne sous la forme d'un nuage de points, pour toutes les paires possibles. Ce type de graphe est dit *scatter matrix* ou encore *splom* (*Scatter PLOt Matrix*).

Les fréquences d'attribution sont propres à chaque prénom, donc on conserve seulement une ligne par prénom (pour éviter de tracer plusieurs fois le même point).

Le graphe est de taille assez importante, nous allons donc plutôt l'enregistrer dans un fichier HTML. Après l'exécution de la cellule ci-dessous, ouvrez le fichier `splom.html` à la racine du répertoire du TD2.

In [32]:
import plotly.express as px

fig = px.scatter_matrix(df_freq_only.drop_duplicates(subset='name'), 
                        dimensions=freq_cols,
                        hover_name='prenom')

fig.write_html('splom.html')

ModuleNotFoundError: No module named 'plotly'

Que peut-on lire sur ce graphe ?

Et bien pas grand chose. Il n'y a visiblement pas particulièrement de colonnes qui apparaissent nettement comme corrélées. Néanmoins, vu le nombre d'information dans cette matrice de graphes, il est facile de se perdre. Nous allons donc passer outre les données brutes et **utiliser des indicateurs numériques de corrélation**.

Toutes les fréquences d'attribution étant quantitatives, nous pouvons calculer le **coefficient de corrélation de Pearson**. pandas propose une méthode pour calculer rapidement ce coefficient pour chaque paire de colonne.

In [ ]:
corrs = df_freq_only.corr(method='pearson')

corrs

Affichons désormais une représentation graphique de cette matrice de corrélation. Les *heatmap* sont particulièrement bien adaptées pour cette application

In [ ]:
import plotly.graph_objects as go

go.Figure(data=go.Heatmap(z=corrs, x=freq_cols, y=freq_cols))

Comme prévu, la plupart des colonnes ne sont pas corrélées deux à deux (coefficient proche de 0). La seule corrélation assez forte détectée est entre les fréquences `europe_nord` et `asie_est` ce qui semble assez curieux.

#### Méthodes factorielles

Une deuxième façon d'identifier les colonnes redondantes consiste à trouver les **combinaisons de colonnes présentant le plus de variance** (au sens statistique du terme). C'est le principe des méthodes factorielles.

Nous allons effectuer une **analyse en composantes principales** sur les 8 colonnes de fréquences. Nous obtiendrons en sortie les composantes dites principales, qui n'ont **pas de sens physique** mais qui **"condensent" le mieux les données**, ainsi que l'**importance de chacune de ces composantes principales**.

Puisque nous commençons la PCA avec 8 colonnes, nous obtiendrons 8 composantes principales en sortie. Néanmoins, certaines de ces composantes principales peuvent avoir un poids bien plus faible que d'autres sur la variance des données. En d'autres termes, ces **composantes "faibles" ne rajoutent que peu d'information supplémentaire** au jeu de données et **peuvent être éliminées** sans pour autant perdre trop d'information.

L'algorithme de PCA est implémentée dans la bibliothèque `scikit-learn` que vous retrouverez probablement pendant le cours de Machine Learning. Nous n'allons pas s'attarder sur l'utilisation de cette bibliothèque et passons droit au but. Pour plus de détails, vous pouvez vous référer à la [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html).

##### Remarques
* L'algorithme de PCA requiert des variables en entrée normalisées, avec des amplitudes et des variantes similaires. En pratique, les fréquences d'attributions vérifient à peu près déjà cette propriété (vérifiez le !), donc nous n'allons pas procéder à une normalisation supplémentaire.
* L'algorithme de PCA ne supporte pas les valeurs manquantes, donc nous allons les remplacer par des `0`. En pratique, toute valeur manquante dans les données d'origine est censéee être une fréquence trop faible pour être mesurée / rapportée, et donc proche de 0.

In [ ]:
from sklearn.decomposition import PCA

pca_model = PCA()
pca_model.fit(X=df_freq_only[freq_cols].fillna(0))

Créons une représentation graphique pour expliciter les composantes principales trouvées d'une part, et le % de variance expliquée par chaque CP d'autres part.

Sur le graphe ci-dessous, vous retrouvez les **8 composantes principales**, classées par ordre de variance expliquée (donc grosso modo **par ordre de quantité d'informations** apportées au jeu de données).

Le pourcentage de variance expliquée est représentée par la courbe, on remarque que la somme de ces pourcentages donne 1 assez logiquement (l'ensemble des données donnent l'ensemble de l'information dans les données...).

Les **8 CP** sont présentées **en abscisse**, et vous observez leur **décomposition selon les variables d'origine** du jeu de donnée **en ordonnée**.

In [ ]:
fig = go.Figure(data=[go.Bar(x=list(range(1, 9)), 
                             y=pca_model.components_[:, i], 
                             name=freq_cols[i])  
                      for i in range(8)],
                layout = go.Layout(bargap=0.25)
               )

fig.add_scatter(x=list(range(1, 9)), y=pca_model.explained_variance_ratio_,
                name='Variance expliquée par la CP')
fig.show()

Les 3 première CP représentent près de **90% de la variance totale** du jeu de données. On pourrait donc ne conserver que ces 3 CP et supprimer les autres colonnes, le jeu de données étant drastiquement réduit dans ce cas.

En réalité, lorsque l'on s'intéresse à la composition de ces CP majeures, on observe qu'elles sont **toutes dominées par une variable du jeu de base** :
* CP 1 dominée par `europe_sud` (donc CP 1 est une image légèrement modifiée de `europe_sud`)
* CP 2 dominée par `europe_ouest` et `France` (mais naturellement `France` et `europe_ouest` sont très proches, donc on aurait plutôt CP 2 dominée par `europe_ouest`)
* CP 3 dominée par `europe_est`

Les variables suivantes sont également dominées par une variable du jeu de base, mais parfois dans une moindre mesure.

Rappelez vous que le jeu de données est **biaisé envers les prénoms européens**. Ces derniers possèdent donc beaucoup plus de données remplies que les prénoms orientaux par exemple, et représentent par conséquent la **majorité des informations contenue** dans le jeu de données.

La PCA a donc simplement mis en évidence le **déséquilibre d'informations entre les différentes zones géographiques**. On pourrait donc à partir de ces résultats ne conserver que les prénoms européens puisque ce sont ceux pour lesquels la majorité de l'information est disponible. Rappelons nous que nous avons par exemple déjà supprimé la zone `Autres` plus tôt lors de l'intégration.

En pratique, **nous conserverons toutes les colonnes** (et donc toutes les zones géographiques), même les moins remplies. En effet, nous souhaitons conserver ce niveau de détail pour l'analyse à venir. De plus, le jeu de données reste relativement réduit et facile à manipuler. S'il avait du mal à tenir en mémoire, nous pourrions envisager la suppression de certaines colonnes, à commencer par `asie_centre` (dernière CP).

#### Réduction de la numérosité

Le nombre de lignes est correct pour l'instant, s'élevant à environ 3 milions d'enregistrements. Il n'est donc pas très utile de réduire davantage ce tableau de données.

----

## 5. Transformation

Nous allons appliquer une série de transformations pour mettre en application les notions du cours.

In [ ]:
# On recharge les données et on applique les types corrects

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

df = pd.read_csv('merged_prenoms.csv', index_col=0, dtype={
    'sexe': 'category',
    'prenom': 'category',
    'departement': 'int',
    'gender': 'category',
})

df = df.astype({'departement': 'category'})

# Remplissage des fréquences manquantes par 0
freq_cols = ['France', 'europe_est', 'europe_nord', 
             'europe_sud', 'europe_ouest', 'asie_ouest', 
             'asie_est', 'asie_centre']
df[freq_cols] = df[freq_cols].fillna(0)

### Généralisation de concept

Nos données sont précises pour les effectifs des naissances au niveau du département. En pratique, il n'est pas forcément utile d'avoir une telle précision pour les analyses à venir, la granularité `région` devrait suffire largement. Nous procédons donc à une **généralisation des modalités de `departement`**. Le nombre de lignes devrait drastiquement diminuer avec l'opération (passage de 101 départements à 16 régions).

In [ ]:
# Connaissez-vous bien vos départements ?
dict_regions = {'Auvergne-Rhône-Alpes': [1, 3, 7, 15, 26, 38, 42, 43, 63, 69, 73, 74],
               'Bourgogne-Franche-Comté' : [21, 25, 39, 58, 70, 71, 89, 90],
               'Bretagne' : [22, 29, 35, 56],
               'Centre-Val de Loire': [18, 28, 36, 37, 41, 45],
               'Corse': [20],
               'Grand Est': [8, 10, 51, 52, 54, 55, 57, 67, 68, 88],
               'DOM/TOM': [97, 971, 972, 973, 974, 975, 976],
               'Hauts-de-France': [2, 59, 60, 62, 80],
               'Ile-de-France': [75, 77, 78, 91, 92, 93, 94, 95],
               'Normandie': [14, 27, 50, 61, 76],
               'Nouvelle-Acquitaine': [16, 17, 19, 23, 24, 33, 40, 47, 64, 79, 86, 87],
               'Occitanie': [9, 11, 12, 30, 31, 32, 34, 46, 48, 65, 66, 81, 82],
               'Pays de la Loire': [44, 49, 53, 72, 85],
               'PACA': [4, 5, 6, 13, 83, 84]}

# On inverse le dictionnaire pour avoir dpt: region
dict_dep = dict()
for region, dpts in dict_regions.items():
    for dpt in dpts:
        dict_dep[dpt] = region

In [ ]:
df['region'] = df.departement.map(dict_dep).astype('category')
df.head()

In [ ]:
df_by_region = df.groupby(['prenom', 'sexe', 'annee', 'region',
                           'name', 'gender', 'France', 'europe_est', 
                           'europe_ouest', 'europe_nord', 'europe_sud',
                           'asie_ouest', 'asie_centre', 'asie_est'], 
                          observed=True).nombre.sum().reset_index()

print(len(df_by_region))
df_by_region.head()

Nous avons ainsi divisé par 3 le nombre de lignes dans le tableau de données.

### Calcul des fréquences 

Afin de permettre des comparaisons équitables dans le temps et tenir compte de la croissance démographique française au cours du XXe siècle, nous allons normaliser chaque ligne du tableau par le nombre total de naissances (homme et femme) dans la région concernée à l'année courante.

Autrement dit, pour chaque couple `(annee, departement)`, on souhaite normaliser chaque effectif dans le groupe par l'effectif total du groupe.

In [ ]:
# Deux possibilités pour cette normalisation :
#  1. Faire un groupby et dériver les totaux des effectifs dans chaque groupe, puis fusionner df et le DataFrame des 
#     totaux avec merge()
#  2. Faire un groupby et transformer chaque groupe avec la fonction transform
# On applique la deuxième méthode plus élégante
df_by_region['pourcentage'] = df_by_region.groupby(['annee', 'region'],
                                                   observed=True).nombre.transform(lambda x: x / x.sum())

df_by_region.head()

### Création d'un attribut dérivé

Nous allons certainement vouloir comparer la proximité entre plusieurs prénoms plus tard dans l'analyse. Cette proximité peut être calculée avec une distance lexicographique telle la [distance de Levenshtein](https://fr.wikipedia.org/wiki/Distance_de_Levenshtein), néanmoins ces distances sont souvent très dépendantes du nombre de caractères dans les deux chaînes à comparer. La distance entre "Noa" et "Noah" sera donc radicalement différente de celle entre "Martine" et "Marthine". 

Nous allons donc préférer appliquer une **transformation dite phonétique** sur chaque prénom du jeu de donnée. Ces transformations ont pour but de fournir un code simple censé représenter la prononciation de la chaîne passée en paramètre. Ces transformations sont surtout efficaces pour des données issues du langage naturel (difficile de définir la phonétique d'un mot n'existant pas une langue).

Etant donné que nos données sont principalement en langue française, nous devons sélectionner un algorithme capable de prendre en compte les règles de prononciation du français. L'algorithme [Double Metaphone](https://fr.wikipedia.org/wiki/Double_metaphone) abordé plus haut semble être un bon candidat. Il est implémenté dans le fichier `metaphone.py` situé dans le même dossier que ce notebook.

In [ ]:
from metaphone import dm

print(dm('Michel'))
print(dm('Michèle'))

L'algorithme Double Metaphone donne entre 1 et 2 codes phonétiques représentant la chaîne passée en paramètre. Le premier code est la prononciation classique, le deuxième code (si existant) est une prononciation alternative courante.

Pour le cas de `Michel`, les deux prononciations déterminées sont donc "Mi**ch**el" (avec le "ch" de "cheminée") et "Mi**k**el" (version catalane).

Appliquons cette transformation à tous les prénoms du jeu de données (sauf `_PRENOMS_RARES`). Nous conservons uniquement la prononciation la plus courante pour chacun des prénoms.

In [ ]:
df_by_region['prononciation'] = (df_by_region[df_by_region.prenom != '_PRENOMS_RARES']
                                 .prenom.astype(str)
                                 .apply(lambda x: dm(x)[0]))

df_by_region.head()

Notre travail de transformation du jeu de données étant terminé, nous pouvons enfin enregister une copie des données prétraitées.

In [ ]:
df_by_region.to_csv('prenoms_transformed.csv')

----

## Conclusion

Nous avons pu observer tout au long de ce TD les **difficultés courantes** qui surviennent lors de la préparation de données réelles en vue d'une analyse. Ces étapes de prétraitement sont **longues** et **laborieuses** mais cruciales afin de **faciliter la fouille des données**. En outre, en garantissant une certaine qualité des données, vous pourrez également avoir **davantage confiance dans vos résultats**.

Les différentes opérations ont été effectuées principalement avec `pandas` dans ce TD, mais ce n'est bien évidemment pas le seul outil à même de remplir ces tâches. En particulier, manipuler des données dans une solution de **Data Warehouse orienté colonnes** (type BigQuery, Clickhouse, ...) peut être une **alternative** tout aussi viable, et permettant de travailler sur des données massives contrairement à `pandas`. Ces étapes de *preprocessing* sont **invisibles dans le résultat final** donc les outils employés importent peu tant qu'ils sont bien manipulés et maîtrisés.

----

## Annexe - Le format *tidy data*

Nous avons déjà introduit le format dit ***tidy data*** dans le TD précédent et au début de ce TD. Le mot "format" s'entend ici au **sens abstrait** et pas nécessairement informatique du terme (rien à voir avec des formats de fichiers par exemple).

Ce concept a été popularisé par le statisticien [**Hadley Wickham**](http://vita.had.co.nz/papers/tidy-data.pdf) et consiste à adopter une organisation unique pour toutes les **données tabulaires** en obéissant à **trois règles** simples :
* Chaque **colonne** contient une unique **variable**
* Chaque **ligne** contient une unique **observation** pour chacune des variables
* Chaque **cellule** du tableau contient une **valeur unique**

<img src='https://images.squarespace-cdn.com/content/v1/5b872f96aa49a1a1da364999/1572008171822-KJ0300DR4KCHW8NUUN1N/ke17ZwdGBToddI8pDm48kP0H4u0KUkchoILChBGMIUUUqsxRUqqbr1mOJYKfIPR7LoDQ9mXPOjoJoqy81S2I8N_N4V1vUb5AoIIIbLZhVYy7Mythp_T-mtop-vrsUOmeInPi9iDjx9w8K4ZfjXt2dskF1CFdD1ghxEpYxbqRasKlXJQr8mv4xxZBDj9ez2c1CjLISwBs8eEdxAxTptZAUg/image.png?format=2500w' />

Ce format présente les avantages suivants :
* Facile d'**inclure** ou d'exclure **des variables** par sélection de colonnes. Ce format est particulièrement bien **adapté aux solutions de Data Warehouse orientés colonnes** et vous permettront d'accélérer considérablement vos requêtes
* Facile d'**ajouter une nouvelle variable** au jeu de donnée, il s'agit uniquement d'une colonne supplémentaire
* Solution homogène **compatible avec tous les outils** d'analyse et de BI. De nombreux outils s'appuient sur ce format pour **mieux comprendre les données** (e.g. Tableau identifie chaque colonne comme une valeur ou une dimension)
* Plus facile à manipuler lors d'une **agrégation** ou d'une transformation
* Plus facile à **représenter graphiquement** (comme illustré par Plotly Express)

Bien que relativement simple comme concept, la plupart des données que vous aurez l'occasion de manipuler ne respecteront pas parfaitement ces règles.

Parmi les cas d'erreurs classiques, on retrouve :
* Le titre d'une colonne contient une information sur la variable (e.g. une colonne `taille plante A` et une colonne `taille plante B`)
* Une même colonne contient plusieurs variables en même temps (e.g. une colonne contient une information sur les hommes de moins de 50 ans)
* Des variables sont présentes à la fois en lignes et en colonnes (cas typique des données issues d'un fichier Excel ou équivalent)

Corriger ces différentes erreurs pour standardiser toutes les sources dans le format *tidy* est **surprenamment difficile**, notamment en SQL lorsqu'il n'existe **pas de fonction adaptée** de type **`TRANSPOSE`** ou **`PIVOT`** dans le SGBD utilisé. `pandas` contient cependant toutes les fonctions nécessaires pour réaliser simplement ces opérations efficacement. Vous trouverez de nombreux **exemples de corrections dans cet** [**article de blog**](https://www.jeannicholashould.com/tidy-data-in-python.html), les jeux de données utilisés étant présent dans le [dépot GitHub associé](https://github.com/nickhould/tidy-data-python). **S'il vous reste du temps en fin de séance, passez du temps à étudier ces exemples, sinon merci de le faire d'ici la prochaine séance de TD.**

Remarquons enfin que ce format *tidy* n'est pas lié à un quelconque outil ou solution technique. Vous pouvez (et *devriez*) l'**appliquer en toute circonstance et dans tous les contextes**. Lors de vos futurs travaux de modélisation, gardez à l'esprit ces trois règles, vous pourrez ainsi **faciliter** drastiquement **les analyses futures** sur ces données et donc leur **valorisation**.

## Annexe 2 - Jeux de données d'entraînement

Vous pouvez vous entraîner à nettoyer et traiter des données sur les jeux suivants en Open Data. La liste est issue de [cet article](https://makingnoiseandhearingthings.com/2018/04/19/datasets-for-data-cleaning-practice/) et est reproduite ci-dessous par simplicité. Vous trouverez également dans cet article les différents problèmes majeurs de chacun de ces jeux de données.

* [Hourly Weather Surface – Brazil (Southeast region)](https://www.kaggle.com/PROPPG-PPG/hourly-weather-surface-brazil-southeast-region)
* [PhyloTree Data](https://www.kaggle.com/leipzig/phylotree)
* [International Comprehensive Ocean-Atmosphere Data Set](http://icoads.noaa.gov/)
* [CLEANEVAL: Development dataset](https://cleaneval.sigwac.org.uk/)
* [London Air](https://www.londonair.org.uk/london/asp/datadownload.asp)
* [SO MUCH CANDY DATA, SERIOUSLY](http://www.scq.ubc.ca/so-much-candy-data-seriously/)
* [Production and Perception of Linguistic Voice Quality](http://www.phonetics.ucla.edu/voiceproject/voice.html)
* [Australian Marriage Law Postal Survey, 2017](http://www.abs.gov.au/ausstats/abs@.nsf/mf/1800.0)
* [The Metropolitan Museum of Art Open Access](https://github.com/metmuseum/openaccess/)
* [National Drug Code Directory](https://www.fda.gov/Drugs/InformationOnDrugs/ucm142438.htm)
* [Flourish OA](https://github.com/FlourishOA/Data)
* [WikiPlots](https://github.com/markriedl/WikiPlots)
* [Register of UK Parliament Members’ Financial Interests](https://www.parliament.uk/mps-lords-and-offices/standards-and-financial-interests/parliamentary-commissioner-for-standards/registers-of-interests/register-of-members-financial-interests/)
* NYC Gifted & Talented Scores
    * [Résultats 2017-2018](https://docs.google.com/spreadsheets/d/1oJBaH2x369leRtL19HD8n1oZlGqwK9Yz9b14ppUEhXg/edit#gid=439114268)
    * [Résultats 2018-2019](https://docs.google.com/spreadsheets/d/12NoKe86tCH3OZTeqxRNCwRQ5u1ygbfXwXYnqgC6gfwo/edit#gid=420693080)